In [1]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [2]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Osmond',
    last_name='Oke'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Osmond',
 'last_name': 'Oke',
 'client_id': 'OkeOsmond',
 'topic_prefix': 'OkeOsmond'}

Create a consumer without subscribing to any particular topic

In [3]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [4]:
general_consumer.topics()

{'AtmadjaRio-accelerations',
 'AtmadjaRio-locations',
 'DoeJohn-accelerations',
 'DoeJohn-locations',
 'DoeJohn-simple',
 'LoydSam-accelerations',
 'LoydSam-locations',
 'MalenchikAli-locations',
 'OkeOsmond-accelerations',
 'OkeOsmond-locations',
 'PayneBarbara-accelerations',
 'PayneBarbara-locations',
 'SuryaBhushan-accelerations',
 'SuryaBhushan-locations'}

Close the consumer, waiting indefinitely for any needed cleanup.

In [5]:
general_consumer.close()

In [6]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [7]:
consumer.subscription()

{'OkeOsmond-accelerations', 'OkeOsmond-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [ ]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: OkeOsmond-locations
  Partition: 0
  Offset: 0
Message Key: 234cf23b33b64030bde3b75d2cf27546
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: OkeOsmond-locations
  Partition: 0
  Offset: 1
Message Key: 3ec8cf25d0c844a7abad8209b18e9de5
Message Value:
  {
    "key1": "Hey Ozzy",
    "key2": "It is I"
  }

Message metadata:
  Topic: OkeOsmond-locations
  Partition: 0
  Offset: 2
Message Key: 4c0c53def18f40b0993a4038eaf65ece
Message Value:
  {
    "key1": "Countdown to graduation",
    "key2": "30 days away"
  }



Close the consumer, waiting indefinitely for any needed cleanup.

In [ ]:
consumer.close()